In [1]:
import math
import numpy
import torch
import torch.nn as nn
import torchaudio
torchaudio.set_audio_backend("soundfile")
import matplotlib.pyplot as plt

C:\Users\Jokla\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
class AudioSample:
    def __init__(self, filepath):
        loadedData = torchaudio.load(filepath)
        self.waveform = loadedData[0][0]
        self.sampleRate = loadedData[1]
        del loadedData
        self.pitchDeltas = torch.tensor([], dtype = int)
        self.pitchBorders = torch.tensor([], dtype = int)
        self.Pitch = torch.tensor([0], dtype = int)
        self.spectra = torch.tensor([[]], dtype = float)
        self.spectrum = torch.tensor([], dtype = float)
        self.excitation = torch.tensor([], dtype = float) #replace with periodic and aperiodic excitation once implemented
        
    def CalculatePitch(self, expectedPitch, searchRange = 0.2):
        batchSize = math.floor((1. + searchRange) * self.sampleRate / expectedPitch)
        lowerSearchLimit = math.floor((1. - searchRange) * self.sampleRate / expectedPitch)
        batchStart = 0
        while batchStart + batchSize <= self.waveform.size()[0] - batchSize:
            sample = torch.index_select(self.waveform, 0, torch.linspace(batchStart, batchStart + batchSize, batchSize, dtype = int))
            zeroTransitions = torch.tensor([], dtype = int)
            for i in range(lowerSearchLimit, batchSize):
                if (sample[i-1] < 0) and (sample[i] > 0):
                    zeroTransitions = torch.cat([zeroTransitions, torch.tensor([i])], 0)
            error = math.inf
            delta = math.floor(self.sampleRate / expectedPitch)
            for i in zeroTransitions:
                shiftedSample = torch.index_select(self.waveform, 0, torch.linspace(batchStart + i.item(), batchStart + batchSize + i.item(), batchSize, dtype = int))
                newError = torch.sum(torch.pow(sample - shiftedSample, 2))
                if error > newError:
                    delta = i.item()
                    error = newError
            self.pitchDeltas = torch.cat([self.pitchDeltas, torch.tensor([delta])])
            batchStart += delta
        nBatches = self.pitchDeltas.size()[0]
        self.pitchBorders = torch.zeros(nBatches + 1, dtype = int)
        for i in range(nBatches):
            self.pitchBorders[i+1] = self.pitchBorders[i] + self.pitchDeltas[i]
        self.Pitch = torch.mean(self.pitchDeltas.float()).int()
        del batchSize
        del lowerSearchLimit
        del batchStart
        del sample
        del zeroTransitions
        del error
        del delta
        del shiftedSample
        del newError
        del nBatches
        
    def CalculateSpectra(self, iterations = 10, filterWidth = 20):
        Window = torch.hann_window(self.Pitch * 3)
        signals = torch.stft(self.waveform, self.Pitch * 3, hop_length = self.Pitch, win_length = self.Pitch * 3, window = Window, return_complex = True)
        signals = torch.transpose(signals, 0, 1)
        signals = torch.log(signals)
        workingSpectra = signals.abs()
        self.spectra = torch.full_like(workingSpectra, -float("inf"), dtype=torch.float)
        for i in range(iterations):
            workingSpectra = torch.max(workingSpectra, self.spectra)
            self.spectra = workingSpectra
            for i in range(filterWidth):
                self.spectra = torch.roll(workingSpectra, -i, dims = 1) + self.spectra + torch.roll(workingSpectra, i, dims = 1)
            self.spectra = self.spectra / (2 * filterWidth + 1)
        self.spectrum = torch.mean(self.spectra, 0)
        for i in range(self.spectra.size()[0]):
            self.spectra[i] = self.spectra[i] - self.spectrum
        del Window
        del signals
        del workingSpectra
        
    def CalculateExcitation(self):
        Window = torch.hann_window(self.Pitch * 3)
        signals = torch.stft(self.waveform, self.Pitch * 3, hop_length = self.Pitch, win_length = self.Pitch * 3, window = Window, return_complex = True)
        signals = torch.transpose(signals, 0, 1)
        excitations = torch.empty_like(signals)
        for i in range(excitations.size()[0]):
            excitations[i] = signals[i] / (torch.exp(self.spectrum) + torch.exp(self.spectra[i]))
        excitations = torch.transpose(excitations, 0, 1)
        self.excitation = torch.istft(excitations, self.Pitch * 3, hop_length = self.Pitch, win_length = self.Pitch * 3, window = Window, onesided = True)
        del Window
        del signals
        del excitations

In [3]:
class Synthesizer:
    def __init__(self, Excitation, Spectrum, Spectra, SampleRate):
        self.excitation = Excitation
        self.spectrum = Spectrum
        self.spectra = Spectra
        self.sampleRate = SampleRate
        self.returnSignal = torch.tensor([], dtype = float)
        
    def Synthesize(self, pitch, steadiness):
        Window = torch.hann_window(pitch * 3)
        self.returnSignal = torch.stft(self.excitation, pitch * 3, hop_length = pitch, win_length = pitch * 3, window = Window, return_complex = True)
        self.returnSignal = torch.transpose(self.returnSignal, 0, 1)
        for i in range(self.spectra.size()[0]):
            self.returnSignal[i] = self.returnSignal[i] * (torch.exp(self.spectrum) + math.pow(1 - steadiness, 2) * torch.roll(torch.exp(self.spectra), 0, dims=0)[i])
        self.returnSignal = torch.transpose(self.returnSignal, 0, 1)
        self.returnSignal = torch.istft(self.returnSignal, pitch * 3, hop_length = pitch, win_length = pitch * 3, window = Window, onesided=True, )
        del Window
        
    def save(self, filepath):
        torchaudio.save(filepath, torch.unsqueeze(self.returnSignal, 0), self.sampleRate, format="wav", encoding="PCM_S", bits_per_sample=32)

In [4]:
class RelLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(RelLoss, self).__init__()
 
    def forward(self, inputs, targets):    
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        differences = torch.abs(inputs - targets)
        sums = torch.abs(inputs + targets)
        out = (differences / sums).sum() / inputs.size()[0]
        return out

In [5]:
class SpecCrfAi(nn.Module):
    def __init__(self, learningRate=1e-5):
        super(SpecCrfAi, self).__init__()
        
        self.layer1 = torch.nn.Conv2d(1, 10, (51, 3), padding = (25, 0), bias = False)
        self.ReLu1 = nn.ReLU()
        self.layer2 = torch.nn.Conv2d(10, 10, (51, 1), padding = (25, 0), bias = False)
        self.ReLu2 = nn.ReLU()
        self.layer3 = torch.nn.Conv2d(10, 10, (51, 1), padding = (25, 0), bias = False)
        self.ReLu3 = nn.ReLU()
        self.layer4 = torch.nn.Conv2d(10, 1, (51, 1), padding = (25, 0), bias = False)
        
        self.learningRate = learningRate
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learningRate, weight_decay=0.)
        #self.criterion = nn.L1Loss()
        self.criterion = RelLoss()
        
    def forward(self, spectrum1, spectrum2, factor):
        fac = torch.full((spectrum1.size()[0], 1), factor)
        x = torch.cat((spectrum1.unsqueeze(1), fac, spectrum2.unsqueeze(1)), dim = 1)
        x = x.float().unsqueeze(0).unsqueeze(0)
        x = self.layer1(x)
        x = self.ReLu1(x)
        x = self.layer2(x)
        x = self.ReLu2(x)
        x = self.layer3(x)
        x = self.ReLu3(x)
        x = self.layer4(x)
        return x
    
    def processData(self, spectrum1, spectrum2, factor):
        output = torch.squeeze(self(spectrum1, spectrum2, factor))
        return output
    
    def train(self, indata, epochs=1):
        for epoch in range(epochs):
            for data in self.dataLoader(indata):
                spectrum1 = data[0]
                spectrum2 = data[-1]
                indexList = numpy.arange(0, data.size()[0], 1)
                numpy.random.shuffle(indexList)
                for i in indexList:
                    factor = i / float(data.size()[0])
                    spectrumTarget = data[i]
                    output = torch.squeeze(self(spectrum1, spectrum2, factor))
                    loss = self.criterion(output, spectrumTarget)
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
            print('epoch [{}/{}], loss:{:.4f}'
                  .format(epoch + 1, epochs, loss.data))
            
    def dataLoader(self, data):
        return torch.utils.data.DataLoader(dataset=data, shuffle=True)

In [6]:
audioSample1 = AudioSample("Samples/a_a.wav")
audioSample1.CalculatePitch(95.)
audioSample1.CalculateSpectra(iterations = 50)
audioSample1.CalculateExcitation()

audioSample2 = AudioSample("Samples/a_i.wav")
audioSample2.CalculatePitch(95.)
audioSample2.CalculateSpectra(iterations = 50)
audioSample2.CalculateExcitation()

audioSample3 = AudioSample("Samples/a_u.wav")
audioSample3.CalculatePitch(95.)
audioSample3.CalculateSpectra(iterations = 50)
audioSample3.CalculateExcitation()

audioSample4 = AudioSample("Samples/e_a.wav")
audioSample4.CalculatePitch(95.)
audioSample4.CalculateSpectra(iterations = 50)
audioSample4.CalculateExcitation()

audioSample5 = AudioSample("Samples/i_a.wav")
audioSample5.CalculatePitch(95.)
audioSample5.CalculateSpectra(iterations = 50)
audioSample5.CalculateExcitation()

audioSample6 = AudioSample("Samples/u_e.wav")
audioSample6.CalculatePitch(95.)
audioSample6.CalculateSpectra(iterations = 50)
audioSample6.CalculateExcitation()

audioSampleA = AudioSample("Samples/a2.wav")
audioSampleA.CalculatePitch(95.)
audioSampleA.CalculateSpectra(iterations = 50)
audioSampleA.CalculateExcitation()

audioSampleE = AudioSample("Samples/e.wav")
audioSampleE.CalculatePitch(95.)
audioSampleE.CalculateSpectra(iterations = 50)
audioSampleE.CalculateExcitation()

audioSampleI = AudioSample("Samples/i.wav")
audioSampleI.CalculatePitch(95.)
audioSampleI.CalculateSpectra(iterations = 50)
audioSampleI.CalculateExcitation()

audioSampleU = AudioSample("Samples/u.wav")
audioSampleU.CalculatePitch(95.)
audioSampleU.CalculateSpectra(iterations = 50)
audioSampleU.CalculateExcitation()

In [7]:
audioSamples = [audioSample1, audioSample2, audioSample3, audioSample4, audioSample5, audioSample6]
trainSpectra = [0, 0, 0, 0, 0, 0]
for i in range(6):
    trainSpectra[i] = torch.empty_like(audioSamples[i].spectra)
    for j in range(audioSamples[i].spectra.size()[0]):
        trainSpectra[i][j] = audioSamples[i].spectrum + audioSamples[i].spectra[j]
    
specCrfAi = SpecCrfAi()
for i in range(6):
    specCrfAi.train(trainSpectra[i], epochs = 50)

epoch [1/50], loss:0.9967
epoch [2/50], loss:0.9881
epoch [3/50], loss:0.9764
epoch [4/50], loss:0.9581
epoch [5/50], loss:0.9297
epoch [6/50], loss:0.8856
epoch [7/50], loss:0.8185
epoch [8/50], loss:0.6995
epoch [9/50], loss:0.5348
epoch [10/50], loss:0.3388
epoch [11/50], loss:0.1258
epoch [12/50], loss:0.0279
epoch [13/50], loss:0.0131
epoch [14/50], loss:0.0103
epoch [15/50], loss:0.0101
epoch [16/50], loss:0.0101
epoch [17/50], loss:0.0106
epoch [18/50], loss:0.0102
epoch [19/50], loss:0.0104
epoch [20/50], loss:0.0103
epoch [21/50], loss:0.0093
epoch [22/50], loss:0.0093
epoch [23/50], loss:0.0097
epoch [24/50], loss:0.0097
epoch [25/50], loss:0.0088
epoch [26/50], loss:0.0087
epoch [27/50], loss:0.0094
epoch [28/50], loss:0.0087
epoch [29/50], loss:0.0085
epoch [30/50], loss:0.0086
epoch [31/50], loss:0.0089
epoch [32/50], loss:0.0092
epoch [33/50], loss:0.0079
epoch [34/50], loss:0.0079
epoch [35/50], loss:0.0075
epoch [36/50], loss:0.0086
epoch [37/50], loss:0.0079
epoch [38/

In [8]:
synthesizer = Synthesizer(audioSample3.excitation, audioSample3.spectrum, audioSample3.spectra, audioSample3.sampleRate)
synthesizer.Synthesize(audioSample3.Pitch, 1.)
synthesizer.save("Output_Crf.wav")

In [9]:
output = specCrfAi.processData(trainSpectra[1][0], trainSpectra[1][-1], 0.5).detach()

In [10]:
class Synthesizer2:
    def __init__(self, SampleRate):
        self.sampleRate = SampleRate
        self.returnSignal = torch.tensor([], dtype = float)
        
    def Synthesize(self, steadiness, Spectrum, SpectraElement, Excitation):
        #Window = torch.hann_window(pitch * 3)
        modSpectrum = torch.exp(Spectrum) + math.pow(1 - steadiness, 2) * torch.roll(torch.exp(SpectraElement), 0, dims=0)
        modSpectrum = torch.cat((modSpectrum, torch.flip(modSpectrum, [0])[1:-1]), dim = 0)
        result = torch.fft.fft(Excitation) * modSpectrum
        result = torch.fft.ifft(result)
        return result
        
    def SynthesizeBatch(self, steadiness, Spectrum, Spectra, Excitation):
        poles = [0.]
        waveforms = []
        self.returnSignal = torch.zeros_like(Excitation)
        for i in range(Spectra.size()[0])
            delta = (Spectra[i].size()[0] * 2 - 1) / 3
            poles.append(poles[i] + delta)
            ExcitationSegment = Excitation[2 * poles[i] - poles[i+1]:2 * poles[i+1] - poles[i]]
            result = self.Synthesize(steadiness, Spectrum, Spectra[i], ExcitationSegment)
            waveforms.append(result)
            
        #first edgecase
        
        for i in range(1, Spectra.size()[0] - 1)
            pole1 = (poles[i] + poles[i-1]) / 2
            pole2 = (poles[i+1] + poles[i]) / 2
            pole3 = (poles[i+2] + poles[i+1]) / 2
            waveformStart = 2 * poles[i] - poles[i+1]
            waveformEnd = 2 * poles[i+1] - poles[i]
            if waveformStart > pole1:
                for x in range(waveformStart, pole2):
                    AscSlope = torch.pow(torch.sin((x-waveformStart) * 0.5 * math.pi / (pole2-waveformStart)), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-waveformStart] * AscSlope
            elif 2 * poles[i] - poles[i-1] < pole2:
                waveforms[i][0:pole1-waveformStart] = 0.
                for x in range(pole1, 2 * poles[i] - poles[i-1]):
                    AscSlope = torch.pow(torch.sin((x-pole1) * 0.5 * math.pi / (2 * poles[i] - poles[i-1]-pole1)), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-pole1] * AscSlope
            else:
                waveforms[i][0:pole1-waveformStart] = 0.
                for x in range(pole1, pole2):
                    AscSlope = torch.pow(torch.sin((x-pole1) * 0.5 * math.pi / (pole2-pole1)), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-waveformStart] * AscSlope
            if waveformEnd < pole3:
                for x in range(pole2, waveformEnd):
                    DescSlope = torch.pow(torch.cos((x-pole2) * 0.5 * math.pi / (waveformEnd-pole2)), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-waveformStart] * DescSlope
            elif 2 * poles[i+1] - poles[i+2] > pole2:
                waveforms[i][pole3-waveformEnd:-1] = 0.
                for x in range(pole2, pole3):
                    DescSlope = torch.pow(torch.cos((x - 2 * poles[i+1] + poles[i+2]) * 0.5 * math.pi / (pole3 - 2 * poles[i+1] + poles[i+2])), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-waveformStart] * DescSlope
            else:
                waveforms[i][pole3-waveformEnd:-1] = 0.
                for x in range(pole2, pole3):
                    DescSlope = torch.pow(torch.cos((x-pole2) * 0.5 * math.pi / (pole3-pole2)), 2)
                    waveforms[i][x-waveformStart] = waveforms[i][x-waveformStart] * DescSlope
            self.returnSignal[waveformStart:waveformEnd] = self.returnSignal[waveformStart:waveformEnd] + waveforms[i]
            
        #final edgecase
            
    def save(self, filepath):
        torchaudio.save(filepath, torch.unsqueeze(self.returnSignal, 0), self.sampleRate, format="wav", encoding="PCM_S", bits_per_sample=32)